# 통합 파일

## 스키마

# 라이브러리 및 데이터 불러오기

In [ ]:
import pandas as pd 
import numpy as np 
from glob import glob

from urllib.request import urlopen 
import json

import folium

In [ ]:
file = glob("*")
file = sorted(file)
file

In [ ]:
# 편의시설
df_fac = pd.read_csv(file[1], index_col = 0)
df_fac_coin = pd.read_csv(file[2], index_col = 0)
df_fac_pharm = pd.read_csv(file[3], index_col = 0)
df_fac_store = pd.read_csv(file[4], index_col = 0)

In [ ]:
# 녹지
df_green = pd.read_csv(file[5], index_col = 0)

In [ ]:
# 치안
df_safe = pd.read_csv(file[6], index_col = 0)
df_safe_entertain = pd.read_csv(file[7], index_col = 0)
df_safe_police = pd.read_csv(file[8], index_col = 0)

In [ ]:
# 교통
df_traffic = pd.read_csv(file[9], index_col = 0)
df_traffic_bus = pd.read_csv(file[10], index_col = 0)
df_traffic_subway = pd.read_csv(file[11], index_col = 0)

In [ ]:
#법정동코드
Statutory_dong_code = pd.read_csv(file[12], index_col = 0)

In [ ]:
#전월세
df_houseprice = pd.read_csv(file[13], index_col = 0, encoding = 'cp949')

df_monthprice= pd.read_csv(file[14], index_col = 0)
df_monthprice_0_10 = pd.read_csv(file[15], index_col = 0)
df_monthprice_11_15 = pd.read_csv(file[16], index_col = 0)
df_monthprice_16_18 = pd.read_csv(file[17], index_col = 0)

df_yearprice= pd.read_csv(file[18], index_col = 0)
df_yearprice_0_10= pd.read_csv(file[19], index_col = 0)
df_yearprice_11_15= pd.read_csv(file[20], index_col = 0)
df_yearprice_16_18= pd.read_csv(file[21], index_col = 0)


# 전월세 출력 함수

## 동 입력 시 전월세 평균 조회

In [ ]:
def dong_price(dong):
    import pandas as pd
    import warnings
    warnings.filterwarnings(action='ignore')

#데이터 가져오고  법정동 코드 합치기  
    price = df_houseprice
    price['법정동코드'] = price['자치구코드'].astype(str) + price['법정동코드'].astype(str)
    price['법정동코드'] = price['법정동코드'].str[0:8]
    price = price.drop(['자치구코드','지번구분코드', '지번구분', '본번','부번','층', '계약일','건축년도','건물명', '계약기간', '신규계약구분','갱신청구권사용','종전보증금','종전임대료'], axis = 1)


        #월세
    bool_month = price['전월세구분'] == '월세'
    month_df = price[bool_month]
    month_df.rename(columns = {'자치구명':'자치구', '법정동명':'동', '임대료(만원)':'월세'}, inplace = True) # 팀원들과 컬럼명 맞추기
    month_df['월세']  = month_df['월세']
    month_df.head()

    bool_area = month_df['임대면적'] < 60  #1인 가구가 살만한 일반적인 최대치를 18평으로 잡았음
    month_df = month_df[bool_area]
    month_df.head()

        #사람들이 일반적으로 이해하기 쉬운 평으로 컬럼을 하나 더 만들어줌
    month_df['평'] = round(month_df['임대면적']/3.3,0)

        #1평당 가격을 지정하여 평당 월세 비용을 구함
    month_df['월세평당가(만원)'] = round(month_df['월세']/month_df['평'], 2)


        #보증금 이상치 제거
    boo1 = month_df['보증금(만원)'] <= 13500
    month_df = month_df[boo1]

        #월세 이상치 제거
    boo2 = month_df['월세'] <= 121
    month_df = month_df[boo2]

        #평구간 나누기
    month_df['평 구간']=pd.cut(month_df['평'],bins=[0,10,15,18],right=True) #right 옵션 True

    month1 = month_df[month_df['평'] <= 10]
    month2 = month_df[(month_df['평'] > 10) & (month_df['평'] <= 15)]
    month3 = month_df[(month_df['평'] > 15) & (month_df['평'] <= 18)]



        # 전세
    bool_year = price['전월세구분'] == '전세'
    year_df = price[bool_year]
    year_df.rename(columns = {'자치구명':'자치구', '법정동명':'동', '보증금(만원)':'전세금'}, inplace = True)# 팀원들과 컬럼명 맞추기
    year_df.head()

    bool_area = year_df['임대면적'] < 60  #1인 가구가 살만한 일반적인 최대치를 18평으로 잡았음
    year_df = year_df[bool_area]
    year_df.head()


        #사람들이 일반적으로 이해하기 쉬운 평으로 컬럼을 하나 더 만들어줌
    year_df['평'] = round(year_df['임대면적']/3.3,0)

        #1평당 가격을 지정하여 평당 전세 비용을 구함
    year_df['전세평당가(만원)'] = round(year_df['전세금']/year_df['평'], 2)
    year_df.head()


        # 이상치 제거
    boo3 = year_df['전세금'] <= 54000
    year_df = year_df[boo3]

        #평수나누기
    year_df['평 구간']=pd.cut(year_df['평'],bins=[0,10,15,18],right=True) #right 옵션 True

    year1 = year_df[year_df['평'] <= 10]
    year2 = year_df[(year_df['평'] > 10) & (year_df['평'] <= 15)]
    year3 = year_df[(year_df['평'] > 15) & (year_df['평'] <= 18)]


    # 월세 size1에 해당하는 보증금 임대료 평당가
    month_size1_price = month1[['전월세구분','동', '보증금(만원)', '월세', '월세평당가(만원)']].groupby(['동']).mean()
    month_size1_price = month_size1_price.reset_index()
    month_size1_price = month_size1_price[month_size1_price['동'] == dong]

    # 월세 size2에 해당하는 보증금 임대료 평당가
    month_size2_price = month2[['전월세구분','동', '보증금(만원)', '월세', '월세평당가(만원)']].groupby(['동']).mean()
    month_size2_price = month_size2_price.reset_index()
    month_size2_price = month_size2_price[month_size2_price['동'] == dong]

    # 월세 size3에 해당하는 보증금 임대료 평당가
    month_size3_price = month3[['전월세구분','동', '보증금(만원)', '월세', '월세평당가(만원)']].groupby(['동']).mean()
    month_size3_price = month_size3_price.reset_index()
    month_size3_price = month_size3_price[month_size3_price['동'] == dong]   




    year_size1_price = year1[['전월세구분','동', '전세금',  '전세평당가(만원)']].groupby(['동']).mean()
    year_size1_price = year_size1_price.reset_index()
    year_size1_price = year_size1_price[year_size1_price['동'] == dong]


    year_size2_price = year2[['전월세구분','동', '전세금', '전세평당가(만원)']].groupby(['동']).mean()
    year_size2_price = year_size2_price.reset_index()
    year_size2_price = year_size2_price[year_size2_price['동'] == dong]

    year_size3_price = year3[['전월세구분','동', '전세금', '전세평당가(만원)']].groupby(['동']).mean()
    year_size3_price = year_size3_price.reset_index()
    year_size3_price = year_size3_price[year_size3_price['동'] == dong]

    df = pd.merge(month_size1_price, year_size1_price,  how='left', on = "동")
    df2= pd.merge(month_size2_price, year_size2_price,  how='left', on = "동")
    df3= pd.merge(month_size3_price, year_size3_price,  how='left', on = "동")
    g = pd.concat([df,df2,df3])
    g['평수'] = ['0-10','11-15','16-18']
    g = g[['평수', '동', '보증금(만원)', '월세', '월세평당가(만원)', '전세금', '전세평당가(만원)']]
    return g

In [ ]:
dong_price('대치동')

## 원하는 금액 입력 시 해당 동 출력

In [ ]:
def price_mean():
     #''' 원하는 평당가 입력시에 평당가의 평균가 인근 동 출력하기 '''
        
    print('원하는 평당가의 범위 입력 시에 해당 평당가 범위 안에 평균 평당가가 들어있는 동을 출력')   
        
    #전세/월세 중 1 선택
    month_or_year = input('전세/월세 중 1 선택: ')
    #개인이 원하는 금액대 저정하기
    a = int(input('원하는 보증금의 평당가 최소값을 입력하세요(단위: 만원): '))
    b = int(input('원하는 보증금의 평당가 최대값을 입력하세요(단위: 만원): '))
    
    if month_or_year == '월세':
        c = int(input('원하는 월세의 최소값을 입력하세요(단위: 만원): '))
        d = int(input('원하는 월세의 최대값을 입력하세요(단위: 만원): '))

    
    import pandas as pd
    import warnings
    warnings.filterwarnings(action='ignore')

#데이터 가져오고  법정동 코드 합치기  
    price = df_houseprice
    price['법정동코드'] = price['자치구코드'].astype(str) + price['법정동코드'].astype(str)
    price['법정동코드'] = price['법정동코드'].str[0:8]
    price = price.astype({'법정동코드':'int'})
    price["주소"] = price["자치구명"]+ price["법정동명"]
    price = pd.merge(price, Statutory_dong_code, how ="outer", on='주소')
    price = price[[ '법정동코드_y','전월세구분', '임대면적', '보증금(만원)', '임대료(만원)', '건물용도', '자치구', '동']]
    price.rename(columns={'법정동코드_y':"법정동코드"}, inplace=True)
    price = price.dropna(axis=0)
    price

        #월세
    bool_month = price['전월세구분'] == '월세'
    month_df = price[bool_month]
    month_df.rename(columns = {'자치구명':'자치구', '법정동명':'동', '임대료(만원)':'월세'}, inplace = True) # 팀원들과 컬럼명 맞추기
    month_df['월세']  = month_df['월세']
    month_df.head()

    bool_area = month_df['임대면적'] < 60  #1인 가구가 살만한 일반적인 최대치를 18평으로 잡았음
    month_df = month_df[bool_area]
    month_df.head()

        #사람들이 일반적으로 이해하기 쉬운 평으로 컬럼을 하나 더 만들어줌
    month_df['평'] = round(month_df['임대면적']/3.3,0)

        #1평당 가격을 지정하여 평당 월세 비용을 구함
    month_df['월세평당가(만원)'] = round(month_df['월세']/month_df['평'], 2)

    month_df['보증금평당가(만원)'] = round(month_df['보증금(만원)']/month_df['평'], 2)


        #보증금 이상치 제거
    boo1 = month_df['보증금(만원)'] <= 13500
    month_df = month_df[boo1]

        #월세 이상치 제거
    boo2 = month_df['월세'] <= 121
    month_df = month_df[boo2]




        # 전세
    bool_year = price['전월세구분'] == '전세'
    year_df = price[bool_year]
    year_df.rename(columns = {'자치구명':'자치구', '법정동명':'동', '보증금(만원)':'전세금'}, inplace = True)# 팀원들과 컬럼명 맞추기
    year_df.head()

    bool_area = year_df['임대면적'] < 60  #1인 가구가 살만한 일반적인 최대치를 18평으로 잡았음
    year_df = year_df[bool_area]
    year_df.head()


        #사람들이 일반적으로 이해하기 쉬운 평으로 컬럼을 하나 더 만들어줌
    year_df['평'] = round(year_df['임대면적']/3.3,0)

        #1평당 가격을 지정하여 평당 전세 비용을 구함
    year_df['전세평당가(만원)'] = round(year_df['전세금']/year_df['평'], 2)
    year_df.head()


        # 이상치 제거
    boo3 = year_df['전세금'] <= 54000
    year_df = year_df[boo3]
    
    
    
    #전/월세 중에 출력
    if month_or_year == '월세':
        month_result = month_df.groupby(['법정동코드','자치구', '동']).mean()
        month_bool = (month_result['보증금평당가(만원)'] >= a) & (month_result['보증금평당가(만원)'] <= b)
        month_bool2 = (month_result['월세평당가(만원)'] >= c) & (month_result['월세평당가(만원)'] <= d)
        month_result = month_result[month_bool & month_bool2]
        month_result.reset_index(inplace = True)
        return month_result[['법정동코드','자치구', '동', '보증금평당가(만원)', '월세평당가(만원)']]
    
    else:
        year_df = year_df.groupby(['법정동코드','자치구', '동']).mean()
        year_bool = (year_df['전세평당가(만원)'] >= a) & (year_df['전세평당가(만원)'] <= b)
        year_df = year_df[year_bool]
        year_df.reset_index(inplace = True)
        return year_df[['법정동코드','자치구', '동','전세평당가(만원)']]

In [ ]:
a = price_mean()
a

# 최종 동 추천 함수구현 부분

In [ ]:
def fac(a,b,c) :
    
    lst = [a,b,c]
    
    name = { "코인세탁소" : df_fac_coin, "약국" : df_fac_pharm, "편의점" : df_fac_store}
    sub = { a : 0.5, b : 0.3, c: 0.2 }
            
    name[a]["점수1"] = name[a]["점수"] * sub[a]
    name[b]["점수1"] = name[b]["점수"] * sub[b]
    name[c]["점수1"] = name[c]["점수"] * sub[c]
    


fac("약국", "편의점", "코인세탁소")


In [ ]:
table = pd.merge(Statutory_dong_code,  df_fac_store, how ="outer", on='법정동코드')
table1 = pd.merge(df_fac_pharm, df_fac_coin, how="outer", on='법정동코드')
df_fac_score = pd.merge(table, table1, how="outer", on='법정동코드')


df_fac_score['점수'] = df_fac_score['점수1'].fillna(0) + df_fac_score['점수1_x'].fillna(0) + df_fac_score['점수1_y'].fillna(0)
df_fac_score = df_fac_score[["법정동코드" ,"자치구_x_x","동_x_x","점수"]] 
df_fac_score.rename(columns={"자치구_x_x":"자치구","동_x_x":"동"}, inplace=True)
df_fac_score

In [ ]:
def traffic(a,b) :
    
    lst = [a,b]
    
    name = { "지하철" : df_traffic_subway, "버스" : df_traffic_bus}
    sub = { a : 0.6, b : 0.4 }
            
    name[a]["점수1"] = name[a]["점수"] * sub[a]
    name[b]["점수1"] = name[b]["점수"] * sub[b]

    


traffic("지하철","버스")


In [ ]:
table = pd.merge(Statutory_dong_code,  df_traffic_bus, how ="outer", on='법정동코드')
df_traffic_score = pd.merge(table, df_traffic_subway, how="outer", on='법정동코드')

df_traffic_score['점수'] = df_traffic_score['점수1_x'].fillna(0) + df_traffic_score['점수1_y'].fillna(0)
df_traffic_score = df_traffic_score[["법정동코드" ,"자치구_x","동_x","점수"]] 
df_traffic_score.rename(columns={"자치구_x":"자치구","동_x":"동"}, inplace=True)
df_traffic_score

In [ ]:
def safe(a,b) :
    
    lst = [a,b]
    
    name = { "경찰서" : df_safe_police, "유흥업소" : df_safe_entertain,}
    sub = { a : 0.5, b : 0.5 }
            
    name[a]["점수1"] = name[a]["점수"] * sub[a]
    name[b]["점수1"] = name[b]["점수"] * sub[b]

    


safe("경찰서","유흥업소")


In [ ]:
table = pd.merge(Statutory_dong_code,  df_safe_police, how ="outer", on='법정동코드')
df_safe_score = pd.merge(table, df_safe_entertain, how="outer", on='법정동코드')

df_safe_score['점수'] = df_safe_score['점수1_x'].fillna(0) + df_safe_score['점수1_y'].fillna(0)
df_safe_score = df_safe_score[["법정동코드" ,"자치구_x","동_x","점수"]] 
df_safe_score.rename(columns={"자치구_x":"자치구","동_x":"동"}, inplace=True)
df_safe_score

In [ ]:
def rast(a,b,c,d) :
    
    lst = [a,b,c,d]
    
    name = { "편의시설" : df_fac_score, "교통" : df_traffic_score, "녹지" : df_green, "치안" :df_safe_score}
    sub = { a : 0.4, b : 0.3, c : 0.2, d : 0.1  }
            
    name[a]["점수1"] = name[a]["점수"] * sub[a]
    name[b]["점수1"] = name[b]["점수"] * sub[b]
    name[c]["점수1"] = name[c]["점수"] * sub[c]
    name[d]["점수1"] = name[d]["점수"] * sub[d]
    


rast("치안","교통","녹지" ,"편의시설")


In [ ]:
table = pd.merge(Statutory_dong_code,  df_traffic_score, how ="outer", on='법정동코드')
table1 = pd.merge(df_green, df_fac_score, how="outer", on='법정동코드')
table2 = pd.merge(table, table1, how="outer", on='법정동코드')
table2['점수'] = table2['점수1'].fillna(0) + table2['점수1_x'].fillna(0) + table2['점수1_y'].fillna(0)
table2 = table2[["법정동코드" ,"자치구_x_x","동_x_x","점수"]] 
table2.rename(columns={"자치구_x_x":"자치구","동_x_x":"동"}, inplace=True)
table2
rast_score = pd.merge(table2, df_safe_score, how="outer", on='법정동코드')
rast_score['점수'] = rast_score['점수_x'].fillna(0) + rast_score['점수1'].fillna(0) 
rast_score = rast_score[["법정동코드" ,"자치구_x","동_x","점수"]] 
rast_score.rename(columns={"자치구_x":"자치구","동_x":"동"}, inplace=True)
rast_score

In [ ]:
result = pd.merge(a,rast_score, how ="left", on='법정동코드')
result = result.drop(["자치구_y","동_y"],axis=1)
result.rename(columns={"자치구_x":"자치구","동_x":"동"}, inplace=True)
result = result.dropna(axis=0)
result

In [ ]:
result_1=result.nlargest(10,"점수",keep='all')

# 시각화

## Geo JSON 불러오기

In [ ]:
seoul_geo_url = "https://raw.githubusercontent.com/choiimingue/seoul_map/main/EMD_SEOUL.json"
print(seoul_geo_url)

In [ ]:
# 서울 시도 GeoJSON 
with urlopen(seoul_geo_url) as response:
    seoul_geojson = json.load(response)

seoul_geojson["features"][0]["properties"]

## 시각화 샘플

In [ ]:
# 법정동코드 코인세탁소 합계 시각화
f_map_coin = folium.Map([37.541, 126.986], zoom_start=12)

folium.Choropleth(
    geo_data=seoul_geojson,
    name='choropleth',
    data=df_fac_coin,
    columns=["법정동코드",'합계'],
    key_on='feature.properties.EMD_CD',
    fill_color='Reds',
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='동별 코인세탁소 수'
).add_to(f_map_coin)

f_map_coin

## 구분별 데이터 시각화 함수 구현
### get_geo(a)

In [ ]:
# 세부구분 데이터 시각화
def get_geo(a):
    
    # 개별 데이터 지도 시각화 함수
    # 파라미터를 입력
    # a에 세부구분 넣어 지도에 시각화
    
    f_map_fac = folium.Map([37.541, 126.986], zoom_start=11, tiles="cartodbpositron")
    lst = [a]
    for i in lst :
        
        name = { "코인세탁소" : df_fac_coin, "약국" : df_fac_pharm, "편의점" : df_fac_store, 
               "녹지" : df_green, "유흥업소" : df_safe_entertain, "경찰관서" : df_safe_police,
               "버스" : df_traffic_bus, "지하철" : df_traffic_subway, "결과" : result, "탑10" : result_1}
        sub = { a : 0.5,}
        
        

        # 코인세탁
        folium.Choropleth(
        geo_data=seoul_geojson,
        name='choropleth',
        data=name[i],
        columns=["법정동코드",'점수'],
        key_on='feature.properties.EMD_CD',
        fill_color='Greens',
        fill_opacity=sub[i],
        line_opacity=0.5,
        legend_name= f'동별 {i} 수'
        ).add_to(f_map_fac)
        

    return f_map_fac

get_geo("약국")

### get_pricegeo(a)

In [ ]:
def get_pricegeo(a):

    
    # 개별 데이터 지도 시각화 함수
    # 파라미터를 입력
    # a에 세부구분 넣어 지도에 시각화
    
    f_map_fac = folium.Map([37.541, 126.986], zoom_start=12)
    lst = [a]
    for i in lst :
        
        name = { "월세0-10" : df_monthprice_0_10, "월세11-15" : df_monthprice_11_15, "월세16-18" : df_monthprice_16_18, 
               "월세전체" : df_monthprice, "전세0-10" : df_yearprice_0_10, "전세10-15" : df_yearprice_11_15,
               "전세16-18" : df_yearprice_16_18, "전세전체" : df_yearprice}
       
        
        sub = { a : 0.5,}

        folium.Choropleth(
        geo_data=seoul_geojson,
        name='choropleth',
        data=name[i],
        columns=["법정동코드",'평당가(만원)'],
        key_on='feature.properties.EMD_CD',
        fill_color='Greens',
        fill_opacity=sub[i],
        line_opacity=0.5,
        legend_name= f'동별 {i} 평당가'
        ).add_to(f_map_fac)
    
    return f_map_fac

get_pricegeo("월세전체")

##  편의시설 우선순위에 따른 시각화 함수 구현

### 점수 비율 차등 부여 함수

In [ ]:
def geo_score(a,b,c) :
    
    f_map_fac = folium.Map([37.541, 126.986], zoom_start=12)
    lst = [a,b,c]
    
    name = { "코인세탁소" : df_fac_coin, "약국" : df_fac_pharm, "편의점" : df_fac_store, 
                   "녹지" : df_green, "유흥업소" : df_safe_entertain, "경찰관서" : df_safe_police,
                   "버스" : df_traffic_bus, "지하철" : df_traffic_subway}
    sub = { a : 0.9, b : 0.6, c: 0.3 }
            
    name[a]["점수"] = name[a]["점수"] * sub[a]
    name[b]["점수"] = name[b]["점수"] * sub[b]
    name[c]["점수"] = name[c]["점수"] * sub[c]
    
    for i in lst :
        
       
        folium.Choropleth(
        geo_data=seoul_geojson,
        name='choropleth',
        data=name[i],
        columns=["법정동코드",'점수'],
        key_on='feature.properties.EMD_CD',
        fill_color='Greens',
        fill_opacity=0.5,
        line_opacity=0.5,
        legend_name= f'동별 {i} 수'
        ).add_to(f_map_fac)

  
    return f_map_fac

geo_score("약국", "편의점", "코인세탁소")


### 불투명도 차등 부여 함수

In [ ]:
def get_geo_op(a,b,c):
    
    # 편의시설 우선순위에 따른 지도 시각화 함수
    # 파라미터를 우선순위순으로 입력
    # 불투명도에 차이를 두어 지도에 시각화
    lst = [a, b, c]
    f_map_fac = folium.Map([37.541, 126.986], zoom_start=12)
    for i in lst :
        
        name = { "코인세탁소" : df_fac_coin, "약국" : df_fac_pharm, "편의점" : df_fac_store, 
               "녹지" : df_green, "유흥업소" : df_safe_entertain, "경찰관서" : df_safe_police,
               "버스" : df_traffic_bus, "지하철" : df_traffic_subway}
        sub = { a : 0.5, b : 0.3, c : 0.1}
        
        

        # 코인세탁
        folium.Choropleth(
        geo_data=seoul_geojson,
        name='choropleth',
        data=name[i],
        columns=["법정동코드",'합계'],
        key_on='feature.properties.EMD_CD',
        fill_color='Greens',
        fill_opacity=sub[i],
        line_opacity=0.5,
        legend_name= f'동별 {i} 수'
        ).add_to(f_map_fac)
    
    return f_map_fac

get_geo_op("약국", "편의점", "코인세탁소")

In [ ]:
result_1.sort_values(["점수"], ascending = False).reset_index(drop=True)

In [ ]:
get_geo("탑10")